In [1]:
from datetime import datetime
import time
import os
from tqdm import tqdm
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np

In [44]:
bug_df = pd.read_csv('./bug/so details.txt')
bug_df.info()
bug_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3636 entries, 0 to 3635
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CreationDate   3636 non-null   object 
 1   Title          3636 non-null   object 
 2   Tags           3636 non-null   object 
 3   Score          3636 non-null   int64  
 4   AnswerCount    3636 non-null   int64  
 5   CommentCount   3636 non-null   int64  
 6   FavoriteCount  702 non-null    float64
 7   w2v_title      3636 non-null   object 
 8   w2v_tag        3636 non-null   object 
 9   title+tag      3636 non-null   object 
 10  w2v_title+tag  3636 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 312.6+ KB


,CreationDate,Title,Tags,Score,AnswerCount,CommentCount,FavoriteCount,w2v_title,w2v_tag,title+tag,w2v_title+tag
0,2012-07-10T14:09:47.123,How to enable the Java keyword assert in Eclip...,java eclipse assert assertions,64,6,1,8.0,"[0.24032320082187653, 0.42651060223579407, 0.4...","[-0.11689873645082116, 0.3525516837835312, 0.1...",How to enable the Java keyword assert in Eclip...,"[0.24032320082187653, 0.8867842555046082, 0.59..."
1,2012-10-08T12:46:21.523,Stack program in Java without using built-in c...,java data-structures,0,3,4,NaN,"[0.24032320082187653, 0.42886120080947876, 0.3...","[0.1382400542497635, -0.023948220536112785, -0...",Stack program in Java without using built-in c...,"[0.24032320082187653, 0.42886120080947876, 0.3..."


In [45]:
non_bug_df = pd.read_csv('./non_bug/so details.txt')
non_bug_df.info()
non_bug_df.head(2)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17986 entries, 0 to 17985
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CreationDate   17986 non-null  object 
 1   Title          17986 non-null  object 
 2   Tags           17986 non-null  object 
 3   Score          17986 non-null  int64  
 4   AnswerCount    17986 non-null  int64  
 5   CommentCount   17986 non-null  int64  
 6   FavoriteCount  3427 non-null   float64
 7   w2v_title      17986 non-null  object 
 8   w2v_tag        17986 non-null  object 
 9   title+tag      17986 non-null  object 
 10  w2v_title+tag  17986 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 1.5+ MB


,CreationDate,Title,Tags,Score,AnswerCount,CommentCount,FavoriteCount,w2v_title,w2v_tag,title+tag,w2v_title+tag
0,2018-08-06T01:49:50.053,Inconsistency in Java's compareTo method on St...,java comparator,0,0,3,NaN,"[0.6235586404800415, 0.0, 0.15243612229824066,...","[0.007734555751085281, -0.09128731675446033, -...",Inconsistency in Java's compareTo method on St...,"[0.6235586404800415, 0.04729170724749565, 0.15..."
1,2018-08-06T01:49:50.053,Inconsistency in Java's compareTo method on St...,java comparator,0,0,3,NaN,"[0.6235586404800415, 0.0, 0.15243612229824066,...","[0.007734555751085281, -0.09128731675446033, -...",Inconsistency in Java's compareTo method on St...,"[0.6235586404800415, 0.04729170724749565, 0.15..."


In [46]:
from nltk.corpus import stopwords
import nltk
from gensim.models import word2vec
w2v_model = word2vec.Word2Vec.load('./word2vec.model')


def mean_vector(words):
    vec = np.zeros(300).reshape((1, 300))
    for word in words:
        try:
            vec += w2v_model[word].reshape((1, 300))
        except KeyError:
            continue
    return_vec = vec/len(words)
    return_vec = return_vec.reshape((300,))
    return return_vec.tolist()


def max_vector(words):
    vec = np.zeros(300).reshape((1, 300))
    for word in words:
        try:
            vec = np.concatenate((vec, w2v_model[word].reshape((1, 300))),axis=0)
        except KeyError:
            continue
    return_vec = vec.max(axis=0)
    return return_vec.tolist()



def separate_sentence(text):
    remove_chars = '[·’!"\#$%&\'()＃！（）*+,-./:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+'
    text = re.sub(remove_chars, "", text)

    disease_List = nltk.word_tokenize(text)

    filtered = [w for w in disease_List if(w not in stopwords.words('english'))]

    Rfiltered =nltk.pos_tag(filtered)

    filter_word = [i[0] for i in Rfiltered]
    return filter_word


def return_w2v(text):
    words_list = separate_sentence(text)
    # vec = max_vector(words_list)
    vec = mean_vector(words_list)
    return vec


In [47]:
bug_df['w2v_title'] = bug_df['Title'].apply(lambda x: return_w2v(x))

D:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [48]:
bug_df['w2v_tag'] = bug_df['Tags'].apply(lambda x: return_w2v(x))

D:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [49]:
# bug_df['title+tag'] = bug_df['Title']+pd.Series([' ']*len(bug_df))+bug_df['Tags']
# bug_df['w2v_title+tag'] = bug_df['title+tag'].apply(lambda x: return_w2v(x))

In [50]:
bug_df.to_csv('./bug/so details.txt',index=False)

In [28]:
bug_df.iloc[0]['title+tag']


'How to enable the Java keyword assert in Eclipse program-wise? java eclipse assert assertions'

In [51]:
non_bug_df['w2v_title'] = non_bug_df['Title'].apply(lambda x: return_w2v(x))

D:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [52]:
non_bug_df['w2v_tag'] = non_bug_df['Tags'].apply(lambda x: return_w2v(x))

D:\ML_ENVS\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [53]:
# non_bug_df['title+tag'] = non_bug_df['Title']+pd.Series([' ']*len(non_bug_df))+non_bug_df['Tags']
# non_bug_df['w2v_title+tag'] = non_bug_df['title+tag'].apply(lambda x: return_w2v(x))

In [54]:
non_bug_df.to_csv('./non_bug/so details.txt',index=False)

In [55]:
non_bug_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17986 entries, 0 to 17985
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CreationDate   17986 non-null  object 
 1   Title          17986 non-null  object 
 2   Tags           17986 non-null  object 
 3   Score          17986 non-null  int64  
 4   AnswerCount    17986 non-null  int64  
 5   CommentCount   17986 non-null  int64  
 6   FavoriteCount  3427 non-null   float64
 7   w2v_title      17986 non-null  object 
 8   w2v_tag        17986 non-null  object 
 9   title+tag      17986 non-null  object 
 10  w2v_title+tag  17986 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 1.5+ MB


300